In [1]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [4]:
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage für die Stationen
select_value_query = """
    SELECT v.*, b.ort
    FROM value v
    JOIN box_id b ON v.senseid_fk = b.senseid;
"""
cursor.execute(select_value_query)
values = cursor.fetchall()

df = pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

cursor.close()
connection.close()

Verbindung zur Datenbank erfolgreich hergestellt.


Werte Droppen

In [5]:
df = df[df['luftfeuchtigkeit'] != 100.00]
df = df[df['luftfeuchtigkeit'] != 99.90]
df = df[df["pm10"]<1200]
bedingung = df['temperature'] > df['temperature'].quantile(0.99)
df = df.drop(df[bedingung].index)

In [6]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [7]:
stations

array(['Hamburg', 'Mersch', 'Ettlingen', 'Stuttgart', 'München', 'Jena',
       'Berlin', 'Dortmund'], dtype=object)

In [13]:
app = dash.Dash(__name__)

# Layout der Dash-App definieren
app.layout = html.Div(id='app-container', children=[
    html.H1(
        children='Umweltüberwachung Dashboard',
        id='title',
    ),

    html.Button(
        'Toggle Dark/Light Mode',
        id='toggle-button',
        n_clicks=0,
        style={'margin-bottom': '20px'}
    ),

    html.Div(
        children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
        id='description',
    ),

    dcc.Dropdown(
        id='station-dropdown',
        options=stations_options,
        value='all',
    ),

    html.Div(
        children='Wählen Sie eine Kennzahl:',
        id='metric-label',
        style={'marginTop': 20}
    ),
    
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Durchschnitt', 'value': 'mean'},
            {'label': 'Median', 'value': 'median'},
            {'label': 'Varianz', 'value': 'var'},
            {'label': 'Standardabweichung', 'value': 'std'},
            {'label': 'Min', 'value': 'min'},
            {'label': 'Max', 'value': 'max'},
            {'label': 'Boxplot', 'value': 'boxplot'}
        ],
        value='mean',
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),

    html.Div(
        children='Wählen Sie die Aggregationsstufe:',
        id='aggregation-label',
        style={'marginTop': 20}
    ),
    
    dcc.Checklist(
        id='aggregation-checklist',
        options=[
            {'label': 'Jahr', 'value': 'year'},
            {'label': 'Monat', 'value': 'month'},
            {'label': 'Tag', 'value': 'day'},
            {'label': 'Stunde', 'value': 'hour'}
        ],
        value=['year', 'month'],
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),

    html.Div(
        dcc.Graph(id='temperature-graph'),
    ),
    html.Div(
        dcc.Graph(id='luftfeuchtigkeit-graph'),
    ),
    html.Div(
        dcc.Graph(id='pm-graph'),
    )
])

# Funktion zur Berechnung von IQR
def calculate_metrics(group, metric):
    if metric == 'iqr':
        return group.quantile(0.75) - group.quantile(0.25)
    else:
        return group.agg(metric)


# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('temperature-graph', 'figure'),
     Output('luftfeuchtigkeit-graph', 'figure'),
     Output('pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value'),
     Input('aggregation-checklist', 'value')]
)
def update_graphs(selected_station, selected_metric, selected_aggregation):
    # Filtere den DataFrame nach der ausgewählten Station
    if selected_station != 'all':
        filtered_df = df[df['station_name'] == selected_station]
    else:
        filtered_df = df.copy()

    # Überprüfen der gültigen Aggregationskombinationen
    if 'hour' in selected_aggregation and 'day' not in selected_aggregation and "month" in selected_aggregation:
        selected_aggregation.append('day')
    if 'day' in selected_aggregation and 'month' not in selected_aggregation and "year" in selected_aggregation:
        selected_aggregation.append('month')
    if 'hour' in selected_aggregation and "month" not in selected_aggregation and "year" in selected_aggregation:
        selected_aggregation.append("month")

    # Erstelle die Aggregationsspalte basierend auf der Auswahl
    aggregation = ''
    if 'year' in selected_aggregation:
        aggregation += '%Y'
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

    if 'month' in selected_aggregation:
        aggregation += '-%m'
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

    if 'day' in selected_aggregation:
        aggregation += '-%d'
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

    if 'hour' in selected_aggregation:
        aggregation += ' %H'
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
    if aggregation == '-%d':
        filtered_df['aggregation'] = filtered_df['time'].dt.day_of_year

    # Bereinige den DataFrame von NaN-Werten
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    if selected_metric == 'boxplot':
        # Boxplot-Diagramme
        temp_fig = px.box(filtered_df, x='aggregation', y='temperature', title='Boxplot Temperatur')
        humidity_fig = px.box(filtered_df, x='aggregation', y='luftfeuchtigkeit', title='Boxplot Luftfeuchtigkeit')
        pm_fig = px.box(filtered_df, x='aggregation', y=['pm2_5', 'pm10'], title='Boxplot PM2.5 und PM10')
    else:
        # Berechne die Kennzahlen
        daily_df = filtered_df.groupby('aggregation').agg({
            'temperature': selected_metric,
            'luftfeuchtigkeit': selected_metric,
            'pm2_5': selected_metric,
            'pm10': selected_metric
        }).reset_index()

        # Benenne die Aggregationsachse basierend auf den ausgewählten Aggregationsstufen
        agg_label = '-'.join(selected_aggregation).capitalize()

        # Erstelle Diagramme
        temp_fig = px.line(daily_df, x='aggregation', y='temperature', title=f'{selected_metric.capitalize()} Temperatur')
        temp_fig.update_xaxes(title_text=agg_label)
        temp_fig.update_traces(line=dict(color='red'))  # Linie rot machen
        humidity_fig = px.line(daily_df, x='aggregation', y='luftfeuchtigkeit', title=f'{selected_metric.capitalize()} Luftfeuchtigkeit')
        humidity_fig.update_xaxes(title_text=agg_label)
        humidity_fig.update_traces(line=dict(color='red'))  # Linie rot machen
        pm_fig = px.line(daily_df, x='aggregation', y=['pm2_5', 'pm10'], title=f'{selected_metric.capitalize()} PM2.5 und PM10')
        pm_fig.update_xaxes(title_text=agg_label)
        pm_fig.update_traces(line=dict(color='red'))  # Linie rot machen

    return temp_fig, humidity_fig, pm_fig

# Callback zur Umschaltung des Modus
@app.callback(
    [Output('title', 'style'),
     Output('toggle-button', 'style'),
     Output('description', 'style'),
     Output('metric-label', 'style'),
     Output('aggregation-label', 'style'),
     Output('station-dropdown', 'style'),
     Output('metric-radio', 'labelStyle'),
     Output('aggregation-checklist', 'labelStyle'),
     Output('temperature-graph', 'style'),
     Output('luftfeuchtigkeit-graph', 'style'),
     Output('pm-graph', 'style'),
     Output('app-container', 'style')],
    [Input('toggle-button', 'n_clicks')]
)
def toggle_dark_light_mode(n_clicks):
    if n_clicks % 2 == 1:
        dark_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        light_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        dropdown_style = {'backgroundColor': 'black', 'color': 'white'}
        graph_style = {'backgroundColor': 'black'}
        container_style = {'backgroundColor': 'black'}
    else:
        dark_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        light_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        dropdown_style = {'backgroundColor': 'white', 'color': 'black'}
        graph_style = {'backgroundColor': 'white'}
        container_style = {'backgroundColor': 'white'}

    return (dark_mode_style, light_mode_style, dark_mode_style, dark_mode_style,
            dark_mode_style, dropdown_style, light_mode_style, light_mode_style,
            graph_style, graph_style, graph_style, container_style)

# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)